# 6. Interação

_"Um gráfico não é 'desenhado' de uma vez por todas; mas sim 'construído' e reconstruído até que revele todas as relações constituídas pela interação dos dados. As melhores operações gráficas são aquelas performadas pelo próprio tomador de decisões."_ — [Jacques Bertin](https://books.google.com/books?id=csqX_xnm4tcC)

A visualização fornece formas poderosas de analisar os dados. Uma única imagem, no entanto, oferece respostas para, na melhor das hipóteses, um punhado de perguntas. Através da *interação* podemos transformar imagens estáticas em ferramentas para exploração: destacando pontos de interesse, ampliando para revelar padrões refinados e vinculando várias visualizações para explorar relacionamentos multidimensionais.

No cerne da interação está a noção de *seleção*: um meio de indicar ao computador quais elementos ou regiões nos interessam. Por exemplo, podemos passar o mouse sobre um ponto, clicar em múltiplas marcas ou desenhar um retângulo delimitante ao redor de uma região para destacar subconjuntos de dados para análise mais aprofundada.

Junto às codificações visuais e transformações de dados, o Altair oferece uma abstração de *seleção* para interações de autoria. Essas seleções abrangem três aspectos:

1. Manipulação de eventos de entrada para selecionar pontos ou regiões de interesse, como movimentação do mouse, cliques, arrastos, rolagens e toques.

2. Generalizar a partir da entrada para formular uma regra de seleção (ou [*predicado*](https://en.wikipedia.org/wiki/Predicate_%28mathematical_logic%29)) que determine se um registro está ou não dentro de uma seleção.

3. Usar o predicado de seleção para configurar dinamicamente uma visualização ao aplicar _codificações condicionais_, _transformações de filtro_ ou _domínios de escala_.

Esse notebook introduz seleções interativas e explora como utilizá-las para realizar uma variedade de técnicas de interação, tais quais consultas dinâmicas, panorâmica e zoom, detalhes sob demanda, pincelamento e vinculação.

*Esse notebook é parte do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum).*


In [1]:
import pandas as pd
import altair as alt

# 6.1. Conjuntos de Dados

Vamos visualizar uma variedade de conjuntos de dados da coleção [vega-datasets](https://github.com/vega/vega-datasets):
- Um conjunto de dados sobre carros (`cars`) da década de 1970 e início da década de 1980;
- Um conjunto de dados sobre filmes (`movies`), previamente utilizado no notebook de [Transformação de Dados](https://github.com/uwdata/visualization-curriculum/blob/master/altair_data_transformation.ipynb);
- Um conjunto de dados contendo dez anos de preços de ações do [S&P 500](https://en.wikipedia.org/wiki/S%26P_500) (`sp500`);
- Um conjunto de dados sobre ações (`stocks`) de companhias de tecnologia;
- Um conjunto de dados sobre voos (`flights`), incluindo horário de partida, distância e atraso de chegada.

In [2]:
cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

# 6.2. Introduzindo Seleções

Vamos começar com uma seleção básica: simplesmente clicando em um ponto para destacá-lo. Utilizando o conjunto de dados `cars`, vamos começar com um gráfico de dispersão de cavalos de potência versus milhas por galão, com uma codificação de cor para o número de cilindros no motor do carro.

Ademais, criaremos uma instância de seleção chamando `alt.selection_single()`, indicando que queremos uma seleção definida por um *único valor*. Por padrão, a seleção utiliza um clique de mouse para determinar o valor selecionado. Para registrar a seleção com um gráfico, devemos adicionar isso usando o método `.add_selection()`.

Uma  vez que nossa seleção foi definida, podemos utilizá-la como um parâmetro para *codificações condicionais*, que aplicam diferentes codificações dependendo se um registro de dados está dentro ou fora da seleção. Por exemplo, considere o seguinte trecho de código:

~~~ python
color=alt.condition(selection, 'Cylinders:O', alt.value('grey'))
~~~

Essa definição de codificação indica que pontos contidos em `selection` deveriam ser coloridos de acordo com o campo `Cylinder`, enquanto pontos não selecionados deveriam ser coloridos por uma cor padrão `grey` (cinza). Uma seleção vazia inclui *todos* os pontos, e então inicialmente todos os pontos serão coloridos.

*Tente clicar em diferentes pontos no gráfico abaixo. O que acontece? (Clique no fundo para limpar o estado de seleção e retornar a uma seleção "vazia")*

In [4]:
selection = alt.selection_single();

alt.Chart(cars).mark_circle().add_selection(
    selection
).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
)

C:\Users\elian\AppData\Local\Temp\ipykernel_29920\1559093682.py:1: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  selection = alt.selection_single();
C:\Users\elian\AppData\Local\Temp\ipykernel_29920\1559093682.py:3: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  alt.Chart(cars).mark_circle().add_selection(


alt.Chart(...)

É claro, destacar pontos individuais um a um não é particularmente animador! Como veremos, no entanto, seleções de valores individuais fornecem um útil bloco de construção para interações mais poderosas. Ademais, seleções de valores individuais são apenas um dos três tipos fornecidos pelo Altair:

- `selection_single` - seleciona um único valor discreto, por padrão em eventos de clique.
- `selection_multi` - seleciona múltiplos valores discretos. O primeiro valor é selecionado por clique de mouse e valores adicionais são alternados usando shift-click.
- `selection_interval` - seleciona uma gama contínua de valores, inicialmente pelo arrastar do mouse.

Vamos comparar cada um dos tipos de seleções lado a lado. Para manter nosso código compacto vamos primeiro definir uma função (`plot`) que gera uma especificação para gráficos de dispersão assim como o exemplo acima. Podemos passar uma seleção para a função `plot` para aplicá-la ao gráfico:

In [7]:
def plot(selection):
    return alt.Chart(cars).mark_circle().add_selection(
        selection
    ).encode(
        x='Horsepower:Q',
        y='Miles_per_Gallon:Q',
        color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
        opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
    ).properties(
        width=240,
        height=180
    )

Vamos usar a nossa função `plot` para criar três gráficos variantes, um por tipo de seleção.

O primeiro gráfico (`single`) replica nosso exemplo anterior. O segundo gráfico (`multi`) suporta interações shift-click para alternar inclusão de múltiplos pontos dentro da seleção. O terceiro gráfico (`interval`) gera uma região de seleção (ou *pincel*) ao arrastar o mouse. Uma vez criado, você pode arrastar o pincel para selecionar diferentes pontos ou rolar quando o cursor estiver dentro do mesmo para dimensionar (ampliar) o tamanho do pincel.

*Tente interagir com cada um dos gráficos abaixo!*

In [8]:
alt.hconcat(
  plot(alt.selection_single()).properties(title='Single (Click)'),
  plot(alt.selection_multi()).properties(title='Multi (Shift-Click)'),
  plot(alt.selection_interval()).properties(title='Interval (Drag)')
)

C:\Users\elian\AppData\Local\Temp\ipykernel_29920\2299552586.py:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  plot(alt.selection_single()).properties(title='Single (Click)'),
C:\Users\elian\AppData\Local\Temp\ipykernel_29920\1277325702.py:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  return alt.Chart(cars).mark_circle().add_selection(
C:\Users\elian\AppData\Local\Temp\ipykernel_29920\2299552586.py:3: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  plot(alt.selection_multi()).properties(title='Multi (Shift-Click)'),


alt.HConcatChart(...)

Os exemplos acima usam interações padrão (click, shift-click, arrasto) para cada tipo de seleção. Podemos aprofundar na customização de interações fornecendo especificações do evento de input usando a [sintaxe do seletor de evento do Vega](https://vega.github.io/vega/docs/event-streams/). Por exemplo, podemos modificar nossos gráficos `single` e `multi` para ativar em eventos de `mouseover` em vez de eventos de `click`.

*Pressione a tecla shift no segundo gráfico para "pintar" com dados!*

In [9]:
alt.hconcat(
  plot(alt.selection_single(on='mouseover')).properties(title='Single (Mouseover)'),
  plot(alt.selection_multi(on='mouseover')).properties(title='Multi (Shift-Mouseover)')
)

C:\Users\elian\AppData\Local\Temp\ipykernel_29920\861132474.py:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  plot(alt.selection_single(on='mouseover')).properties(title='Single (Mouseover)'),
C:\Users\elian\AppData\Local\Temp\ipykernel_29920\1277325702.py:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  return alt.Chart(cars).mark_circle().add_selection(
C:\Users\elian\AppData\Local\Temp\ipykernel_29920\861132474.py:3: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  plot(alt.selection_multi(on='mouseover')).properties(title='Multi (Shift-Mouseover)')


alt.HConcatChart(...)

Agora que cobrimos as seleções básicas do Altair, vamos fazer um tour pelas várias técnicas de interação que elas permitem!

# 6.3. Consultas Dinâmicas

*Consultas Dinâmicas* permitem explorações rápidas e reversíveis de dados para isolar padrões de interesse. Como definido por [Ahlberg, Williamson & Shneiderman](https://www.cs.umd.edu/~ben/papers/Ahlberg1992Dynamic.pdf), uma consulta dinâmica:

- representa graficamente uma consulta;
- fornece limites visíveis no alcance da consulta;
- fornece uma representação gráfica dos dados e do resultado da consulta;
- dá um feedback imediato do resultado após ajuste da consulta;
- permite novatos a iniciar os trabalhos com pouco treinamento.

Uma abordagem comum é manipular os parâmetros da consulta utilizando widgets de interface de usuário padrão como sliders, botões de opção e menus suspensos. Para gerar widgets de consulta dinâmicos, podemos aplicar uma operação de vinculação de seleção a um ou mais campos de dados que desejamos consultar.

Vamos construir um gráfico de dispersão interativo que utiliza uma consulta dinâmica para filtrar a exibição. Dado um gráfico de dispersão de avaliações de filmes (do Rotten Tomatoes e IMDb) podemos adicionar uma seleção sobre o campo `Major_Genre` que habilite filtrar filmes por gênero.

Para começar, vamos extrair os gêneros únicos (não nulos) dos dados dos filmes:

In [10]:
df = pd.read_json(movies) # carregando data movies
genres = df['Major_Genre'].unique() # escolhendo um único campo de valores
genres = list(filter(lambda d: d is not None, genres)) # filtrando valores não vazios
genres.sort() # ordem alfabética

Para uso posterior, também vamos definir uma lista de valores exclusivos `MPAA_Rating`:

In [11]:
mpaa = ['G', 'PG', 'PG-13', 'R', 'NC-17', 'Not Rated']

Agora criamos uma seleção `single` vinculada a um menu suspenso.

*Use o menu de consulta dinâmica abaixo para explorar os dados. Como as classificações variam por gênero? Como você revisaria o código para filtrar `MPAA_Rating` (G, PG, PG-13, etc.) ao invés de `Major_Genre`?*

In [12]:
selectGenre = alt.selection_single(
    name='Select', # nome da seleção 'Select'
    fields=['Major_Genre'], # limitando a seleção a Major_Genre field
    init={'Major_Genre': genres[0]}, # utilizando a primeira entrada do gênero como valor inicial
    bind=alt.binding_select(options=genres) # ligar a um menu de valores únicos de gênero
)

alt.Chart(movies).mark_circle().add_selection(
    selectGenre
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selectGenre, alt.value(0.75), alt.value(0.05))
)


C:\Users\elian\AppData\Local\Temp\ipykernel_29920\4070697409.py:1: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  selectGenre = alt.selection_single(


TypeError: altair.vegalite.v5.schema.core.SelectionParameter() got multiple values for keyword argument 'value'

Nossa construção acima faz uso de múltiplos aspectos de seleções:

- Demos um nome à seleção (`Select`). Esse nome não é exigido, mas permite que influenciemos os rótulos de texto do menu de consulta dinâmigo gerado. *(O que acontece se você remover o nome? Teste!)*
- Nós restringimos a seleção a um único campo de dados (`Major_Genre`). Anteriomente usamos uma seleção `single`, a seleção era mapeada para pontos de dados individuais. Ao limitar a seleção a um campo específico, podemos selecionar _todos_ os pontos de dados cujo valor do campo `Major_Genre` corresponde ao único valor selecionado.
- Iniciamos `init=...` a seleção para um valor inicial.
- Nós vinculamos (`bind`) a seleção a um widget de interface, neste caso um menu suspenso via `binding_select`.
- Como antes, usamos uma codificação condicional para controlar a opacidade do canal.

# 6.3.1. Vinculando Seções a Entradas Múltiplas

Uma instância de seleção pode ser vinculada a *múltiplos* widgets de consultas dinâmicas. Vamos modificar o exemplo acima para fornecer filtros para *ambos* `Major_Genre` e `MPAA_Rating`, usando botões de opção ao invés de um menu. Nossa seleção `single` agora está definida sobre um único *par* de gêneros e valores de classificação MPAA.

*Procure por conjunções surpreendentes de gênero e classificações indicativas. Há algum filme de terror classificado como G ou PG?*

In [13]:
# Seleção de valor único sobre pares [Major_Genre, MPAA_Rating]
# Utilizar valores específicos fixados como valores iniciais selecionados
selection = alt.selection_single(
    name='Select',
    fields=['Major_Genre', 'MPAA_Rating'],
    init={'Major_Genre': 'Drama', 'MPAA_Rating': 'R'},
    bind={'Major_Genre': alt.binding_select(options=genres), 'MPAA_Rating': alt.binding_radio(options=mpaa)}
)

# scatter plot, modificar a opacidade com base na seleção
alt.Chart(movies).mark_circle().add_selection(
    selection
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selection, alt.value(0.75), alt.value(0.05))
)

C:\Users\elian\AppData\Local\Temp\ipykernel_29920\1549142674.py:3: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  selection = alt.selection_single(


TypeError: altair.vegalite.v5.schema.core.SelectionParameter() got multiple values for keyword argument 'value'

*Fatos curiosos: A classificação PG-13 não existia quando filmes como [Tubarão](https://www.imdb.com/title/tt0073195/) e [Tubarão 2](https://www.imdb.com/title/tt0077766/) foram lançados. O primeiro filme a receber uma classificação PG-13 foi [Amanhecer Violento](https://www.imdb.com/title/tt0087985/), de 1984.*

### Usando Visualizações como Consultas Dinâmicas

Embora os widgets de interface padrão mostrem os valores _possíveis_ para os parâmetros de consulta, eles não visualizam a _distribuição_ desses valores. Também podemos desejar usar interações mais ricas, como seleções de vários valores ou intervalos, em vez de widgets de entrada que selecionam apenas um único valor de cada vez.

Para resolver esses problemas, podemos criar gráficos adicionais para visualizar dados e oferecer suporte a consultas dinâmicas. Vamos adicionar um histograma da contagem de filmes por ano e utilizar uma seleção de intervalo para realçar dinamicamente os filmes em períodos de tempo selecionados.

*Interaja com o histograma de anos para explorar filmes de diferentes períodos de tempo. Vê alguma evidência de [viés de amostragem](https://en.wikipedia.org/wiki/Sampling_bias) ao longo dos anos? (Qual é a relação entre o ano e as classificações dos críticos?)*.

_Os anos vão de 1930 a 2040! Os futuros filmes estão em pré-produção ou  existem erros "fora de um século"? Além disso, dependendo do fuso horário em que estamos, há um pequeno aumento em 1969 ou 1970. Porque isso acontece? (Veja o final do notebook para uma explicação!)_


In [14]:
brush = alt.selection_interval(
    encodings=['x'] # limitar a seleção aos valores do eixo x (ano)
)

# histograma de consulta dinâmico
years = alt.Chart(movies).mark_bar().add_selection(
    brush
).encode(
    alt.X('year(Release_Date):T', title='Films by Release Year'),
    alt.Y('count():Q', title=None)
).properties(
    width=650,
    height=50
)

# scatter plot, modificar a opacidade com base na seleção
ratings = alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(
    width=650,
    height=400
)

alt.vconcat(years, ratings).properties(spacing=5)

C:\Users\elian\AppData\Local\Temp\ipykernel_29920\1468048454.py:6: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  years = alt.Chart(movies).mark_bar().add_selection(


alt.VConcatChart(...)

O exemplo acima fornece consultas dinâmicas utilizando uma _seleção vinculada_ entre gráficos:

- Criamos uma seleção `interval` (`brush`), e definimos `encodings=['x']` para limitar a seleção apenas ao eixo x, resultando num intervalo de seleção unidimensional.
- Registramos o `brush` com o nosso histograma de filmes por ano através de `.add_selection(brush)`.
- Usamos `brush` numa codificação condicional para ajustar a opacidade (`opacity`) do scatter plot.

Esta técnica de interação de selecionar elementos num gráfico e ver os destaques ligados num ou mais gráficos é conhecida como seleção &amp; ligação ([_brushing &amp; linking_](https://en.wikipedia.org/wiki/Brushing_and_linking)).

## Panning &amp; Zooming

O _scatter plot_ da classificação dos filmes é um pouco confuso em alguns sítios, tornando difícil examinar pontos em regiões mais densas. Utilizando as técnicas de interação de _panning_ e _zooming_, podemos inspecionar regiões densas mais de perto.

Comecemos pensando em alguma forma de expressar o _panning_ e o _zooming_ utilizando as seleções do Altair. O que define a "janela de visualização" de um gráfico? _Domínios de escala dos eixos!_

Podemos alterar os domínios de escala para modificar o intervalo visualizado de valores de dados. Para fazer isso interativamente, podemos vincular uma seleção `interval` aos domínios de escala com o código `bind='scales'`. O resultado é que, em vez de uma seleção de intervalo que podemos arrastar e ampliar, podemos arrastar e ampliar toda a área de plotagem!

_No gráfico abaixo, clique e arraste para deslocar (traduzir) a visualização, ou role para ampliar (escalar) a visualização. O que é que você pode descobrir sobre a precisão dos valores de classificação fornecidos?_


In [15]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales')
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # utilizar a extensão mínima para estabilizar a colocação do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

C:\Users\elian\AppData\Local\Temp\ipykernel_29920\2910915221.py:1: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  alt.Chart(movies).mark_circle().add_selection(


alt.Chart(...)

Ao fazer _zooming_, podemos ver que os valores de classificação têm uma precisão limitada! As classificações do Rotten Tomatoes são números inteiros, enquanto as classificações do IMDB são truncadas em décimos. Como resultado, existe uma sobreposição de gráficos mesmo quando fazemos zoom, com vários filmes compartilhando os mesmos valores de classificação.

Lendo o código acima, você pode notar o código `alt.Axis(minExtent=30)` no canal de codificação `y`. O parâmetro `minExtent` garante que uma quantidade mínima de espaço seja reservada para os ticks e rótulos do eixo. Por que fazer isso? Quando fazemos pan e zoom, os rótulos dos eixos podem mudar e fazer com que a posição do título do eixo se desloque. Ao definir uma extensão mínima, podemos reduzir os movimentos de distração no gráfico. Tente alterar o valor de `minExtent`, por exemplo, definindo-o como zero, e depois diminua o zoom para ver o que acontece quando rótulos de eixo mais longos entram na visualização.

O Altair também inclui uma forma abreviada de adicionar _panning_ e _zooming_ a um gráfico. Ao invés de criar diretamente uma seleção, você pode chamar `.interactive()` para que o Altair gere automaticamente uma seleção de intervalos vinculada às escalas do gráfico:


In [16]:
alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # utilizar a extensão mínima para estabilizar a colocação do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
).interactive()

alt.Chart(...)

Por padrāo, as ligações de escala para seleções incluem os canais de codificação `x` e `y`. E se quisermos limitar o _panning_ e o _zoomimg_ ao longo de uma única dimensão? Podemos chamar `encodings=['x']` para restringir a seleção apenas ao canal `x`:


In [17]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales', encodings=['x'])
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # utilizar a extensão mínima para estabilizar a colocação do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

C:\Users\elian\AppData\Local\Temp\ipykernel_29920\722294318.py:1: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  alt.Chart(movies).mark_circle().add_selection(


alt.Chart(...)

_Quando se faz zooming apenas ao longo de um único eixo, a forma dos dados visualizados pode mudar, afetando potencialmente a nossa perceção das relações nos dados. [Escolher uma proporção de aspecto adequada](http://vis.stanford.edu/papers/arclength-banking) é uma preocupação importante do design de visualização!_


## Navegação: Visão Geral + Detalhe

Ao usar _panning_ e _zooming_, ajustamos diretamente a "janela de visualização" de um gráfico. A estratégia de navegação relacionada de _visão geral + detalhe_ utiliza, em vez disso, uma visão geral de exibição para mostrar _todos_ os dados, ao mesmo tempo que suporta seleções pan e zoom em uma exibição de foco separado.

Abaixo temos dois gráficos de área que mostram uma década de flutuações de preços para o índice de ações S&amp;P 500. Inicialmente, os dois gráficos mostram o mesmo intervalo de dados. Clique e arraste no gráfico de visão geral inferior para atualizar a exibição de foco e examinar intervalos de tempo específicos.


In [18]:
brush = alt.selection_interval(encodings=['x']);

base = alt.Chart().mark_area().encode(
    alt.X('date:T', title=None),
    alt.Y('price:Q')
).properties(
    width=700
)

alt.vconcat(
    base.encode(alt.X('date:T', title=None, scale=alt.Scale(domain=brush))),
    base.add_selection(brush).properties(height=60),
    data=sp500
)

C:\Users\elian\AppData\Local\Temp\ipykernel_29920\342876620.py:12: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  base.add_selection(brush).properties(height=60),


alt.VConcatChart(...)

Ao contrário do nosso caso anterior, panning &amp; zooming, aqui não queremos vincular uma seleção diretamente às escalas de um único gráfico interativo. Em vez disso, queremos vincular a seleção a um domínio de escala em _outro_ gráfico. Para isso, atualizamos o canal de codificação `x` do nosso gráfico de foco, definindo a propriedade domínio (`domain`) da escala para fazer referência à nossa seleção `brush`. Se nenhum intervalo for definido (a seleção está vazia), o Altair ignora o _brush_ e usa os dados subjacentes para determinar o domínio. Quando um intervalo de _brush_ é criado, o Altair o utiliza como o domínio da escala para o gráfico de foco.

# 6.6. Detalhes sob demanda

Uma vez que identificamos pontos de interesse em uma visualização, muitas vezes queremos saber mais sobre eles. _Detalhes sob demanda_ refere-se à capacidade de consultar interativamente mais informações sobre valores selecionados. As _tooltips_ são um meio útil para fornecer esses detalhes sob demanda. No entanto, as tooltips geralmente exibem informações apenas para um único ponto de dados por vez. Como podemos mostrar mais?

O gráfico de dispersão das avaliações de filmes inclui vários outliers potencialmente interessantes, onde as classificações do Rotten Tomatoes e do IMDB divergem. Vamos criar um gráfico que nos permita selecionar interativamente pontos e exibir seus rótulos. Para ativar a consulta de filtro tanto na interação de passar o mouse quanto no clique, usaremos o [operador de composição do Altair](https://altair-viz.github.io/user_guide/interactions.html#composing-multiple-selections) `|` ("ou").

_Passe o mouse sobre os pontos no gráfico de dispersão abaixo para ver um destaque e um rótulo de título. Pressione Shift e clique nos pontos para tornar as anotações mais persistentes e visualizar vários rótulos ao mesmo tempo. Quais filmes são amados pelos críticos do Rotten Tomatoes, mas não pelo público geral no IMDB (ou vice-versa)? Veja se consegue encontrar possíveis erros, como dois filmes diferentes com o mesmo nome que foram acidentalmente combinados!_

In [19]:
hover = alt.selection_single(     
    on='mouseover',  # selecionar ao passar o mouse
    nearest=True,    # seleciona o ponto mais próximo do cursor do mouse
    empty='none'     # a seleção vazia não deve corresponder a nada
)

click = alt.selection_multi(
    empty='none' # seleção vazia sem pontos
)

# codificações de gráfico de dispersão compartilhadas por todas as marcas
plot = alt.Chart().mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q'
)

# base comum para novas camadas
base = plot.transform_filter(
    hover | click # filtrar para os pontos de qualquer seleção
)

# pontos do gráfico de dispersão de camadas, anotações de halo e etiquetas de título
alt.layer(
    plot.add_selection(hover).add_selection(click),
    base.mark_point(size=100, stroke='firebrick', strokeWidth=1),
    base.mark_text(dx=4, dy=-8, align='right', stroke='white', strokeWidth=2).encode(text='Title:N'),
    base.mark_text(dx=4, dy=-8, align='right').encode(text='Title:N'),
    data=movies
).properties(
    width=600,
    height=450
)

C:\Users\elian\AppData\Local\Temp\ipykernel_29920\181612101.py:1: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  hover = alt.selection_single(
C:\Users\elian\AppData\Local\Temp\ipykernel_29920\181612101.py:7: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  click = alt.selection_multi(
C:\Users\elian\AppData\Local\Temp\ipykernel_29920\181612101.py:24: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  plot.add_selection(hover).add_selection(click),


alt.LayerChart(...)


O exemplo acima adiciona três novas camadas ao gráfico de dispersão: uma  anotação circular, texto branco para fornecer um fundo legível, e texto preto exibindo o título do filme. Além disso, esse exemplo usa duas seleções em conjunto:

1. Uma seleção única (`hover`), que inclui `nearest=True` para selecionar automaticamente o ponto de dados mais próximo conforme o mouse se move.
2. Uma seleção múltipla (`click`), que permite criar seleções persistentes usando Shift + clique.



Ambas as seleções incluem a configuração `empty='none'` para indicar que nenhum ponto deve ser incluído se a seleção estiver vazia. Essas seleções são então combinadas em um único predicado de filtro — o operador lógico _ou_ entre `hover` e `click` — para incluir os pontos pertencentes a _qualquer uma_ das seleções. Esse predicado é usado para filtrar as novas camadas e exibir anotações e rótulos apenas para os pontos selecionados.

Usando seleções e camadas, podemos criar diversos designs para exibir detalhes sob demanda! Por exemplo, aqui está uma série temporal em escala logarítmica com os preços de ações de tecnologia, anotada com uma linha-guia e rótulos para a data mais próxima do cursor do mouse:

In [20]:
# selecione um ponto para o qual fornecer detalhes sob demanda
label = alt.selection_single(
    encodings=['x'], # limita a seleção ao valor do eixo x
    on='mouseover',  # seleciona ao passar o mouse sobre os pontos
    nearest=True,    # seleciona o ponto de dados mais próximo do cursor
    empty='none'     # seleção vazia inclui nenhum ponto de dados
)

# define o gráfico de linha base dos preços das ações
base = alt.Chart().mark_line().encode(
    alt.X('date:T'),
    alt.Y('price:Q', scale=alt.Scale(type='log')),
    alt.Color('symbol:N')
)

alt.layer(
    base, # gráfico de linha base

    # adiciona uma marca de linha para servir como guia
    alt.Chart().mark_rule(color='#aaa').encode(
        x='date:T'
    ).transform_filter(label),

    # adiciona marcas circulares para os pontos de tempo selecionados, oculta os pontos não selecionados
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    # adiciona texto com contorno branco para fornecer um fundo legível para as anotações
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='price:Q'
    ).transform_filter(label),

    # adiciona rótulos de texto para os preços das ações
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='price:Q'
    ).transform_filter(label),

    data=stocks
).properties(
    width=700,
    height=400
)

C:\Users\elian\AppData\Local\Temp\ipykernel_29920\2048310671.py:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  label = alt.selection_single(
C:\Users\elian\AppData\Local\Temp\ipykernel_29920\2048310671.py:27: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(label),


alt.LayerChart(...)

_Colocando em prática o que aprendemos até agora: você consegue modificar o gráfico de dispersão de filmes acima (aquele com consulta dinâmica ao longo dos anos) para incluir uma marca de linha que exiba a média das avaliações do IMDB (ou Rotten Tomatoes) dentro do intervalo de anos selecionado?_

## Seleções &amp; Ligações, Revisitados

Anteriormente neste trabalho, vimos um exemplo de _seleções &amp; ligações_: usando um histograma de consulta dinâmica para destacar pontos em um gráfico de dispersão de classificação de filmes. Aqui, visitaremos alguns exemplos adicionais envolvendo seleções vinculadas.

Voltando ao conjunto de dados `cars`, podemos utilizar o operador `repeat` para construir uma [matriz de dispersão (SPLOM)](https://en.wikipedia.org/wiki/Scatter_plot#Scatterplot_matrices) que mostra associações entre quilometragem, aceleração e potência. Podemos definir uma seleção `intervalo` e incluí-la _dentro_ da nossa especificação de gráfico de dispersão repetido para permitir seleções associadas entre todos os gráficos.

_Clique e arraste em qualquer um dos gráficos abaixo para executar a seleção &amp; ligação!_

In [21]:
brush = alt.selection_interval(
    resolve='global' # resolver todas as seleções para uma única instância global
)

alt.Chart(cars).mark_circle().add_selection(
    brush
).encode(
    alt.X(alt.repeat('column'), type='quantitative'),
    alt.Y(alt.repeat('row'), type='quantitative'),
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.1))
).properties(
    width=140,
    height=140
).repeat(
    column=['Acceleration', 'Horsepower', 'Miles_per_Gallon'],
    row=['Miles_per_Gallon', 'Horsepower', 'Acceleration']
)

C:\Users\elian\AppData\Local\Temp\ipykernel_29920\3292460631.py:5: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  alt.Chart(cars).mark_circle().add_selection(


alt.RepeatChart(...)

Note acima a utilização de `resolve='global'` na seleção `interval`. A configuração padrão de `'global'` indica que em todos os gráficos apenas uma seleção pode estar ativa por vez. No entanto, em alguns casos, podemos querer definir seleções em múltiplos gráficos e combinar os resultados. Se utilizarmos `resolve='union'`, a seleção será a _união_ de todos as seleções: se um ponto residir em qualquer uma delas, ele será selecionado. Alternativamente, se utilizarmos `resolve='intersect'`, a seleção consistirá na _interseção_ de todos as seleções: apenas os pontos que residem dentro de todas as seleções serão selecionados.

_Tente definir o parâmetro `resolve` como `union` e `intersect` e veja como isso muda a lógica de seleção resultante._

### Filtragem cruzada

Todos os exemplos de seleção &amp; ligação que vimos utilizam codificações condicionais, por exemplo, para alterar os valores de opacidade em resposta a uma seleção. Outra opção é usar uma seleção definida em uma visualização para _filtrar_ o conteúdo de outra visualização.

Vamos construir uma coleção de histogramas para o conjunto de dados voos (_flights_): atraso (_delay_) de chegada (quão cedo ou tarde um vôo chega, em minutos), distância (_distance_) voada (em milhas), e tempo (_time_) de partida (hora do dia). Usaremos o operador `repeat` para criar os histogramas, e adicionaremos uma seleção `interval` para o eixo `x` com seleções via interseção.

Em particular, cada histograma consistirá em duas camadas: uma camada de fundo cinzento e uma camada de primeiro plano azul, com a camada de primeiro plano filtrada pela nossa interseção de seleções. O resultado é uma interação de _filtragem cruzada_ entre os três gráficos!

_Arraste os intervalos das seleções nos gráficos abaixo. À medida que seleciona voos com atrasos de chegada mais longos ou mais curtos, como é que as distribuições de distância e tempo respondem?_

In [22]:
brush = alt.selection_interval(
    encodings=['x'],
    resolve='intersect'
);

hist = alt.Chart().mark_bar().encode(
    alt.X(alt.repeat('row'), type='quantitative',
        bin=alt.Bin(maxbins=100, minstep=1), # até 100 divisórias
        axis=alt.Axis(format='d', titleAnchor='start') # formato inteiro, título alinhado à esquerda
    ),
    alt.Y('count():Q', title=None) # eixo y sem título
)
  
alt.layer(
    hist.add_selection(brush).encode(color=alt.value('lightgrey')),
    hist.transform_filter(brush)
).properties(
    width=900,
    height=100
).repeat(
    row=['delay', 'distance', 'time'],
    data=flights
).transform_calculate(
    delay='datum.delay < 180 ? datum.delay : 180', # atraso de resposta > 3 horas
    time='hours(datum.date) + minutes(datum.date) / 60' # horas fracionadas
).configure_view(
    stroke='transparent' # sem outline
)

C:\Users\elian\AppData\Local\Temp\ipykernel_29920\4166571633.py:15: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  hist.add_selection(brush).encode(color=alt.value('lightgrey')),


alt.RepeatChart(...)

_Ao fazer uma filtragem cruzada, pode-se observar que os vôos atrasados têm maior probabilidade de partir a horas mais tardias. Este fenômeno é frequentemente familiar aos passageiros: um atraso pode propagar-se ao longo do dia, afetando as viagens subsequentes nesse avião. Para ter maior probabilidade de chegar na hora, reserve um vôo cedo!_

A combinação de múltiplas visualizações e seleções interativas pode permitir formas valiosas de raciocínio multidimensional, transformando até histogramas básicos em poderosos dispositivos de entrada para formular perguntas sobre um conjunto de dados!

## Resumo

Para mais informações sobre as opções de interação suportadas no Altair, consulte a [documentação sobre seleção interativa do Altair](https://altair-viz.github.io/user_guide/interactions.html). Para detalhes sobre a personalização de manipuladores de eventos, por exemplo, para compor múltiplas técnicas de interação ou oferecer suporte à entrada por toque em dispositivos móveis, veja a [documentação sobre seleção do Vega-Lite](https://vega.github.io/vega-lite/docs/selection.html).

Interessado em aprender mais?

- A abstração de _seleção_ foi introduzida no artigo [Vega-Lite: Uma Gramática de Gráficos Interativos](http://idl.cs.washington.edu/papers/vega-lite/), de Satyanarayan, Moritz, Wongsuphasawat, &amp; Heer.
- O sistema PRIM-9 (para projeção, rotação, isolamento e mascaramento em até 9 dimensões) é uma das primeiras ferramentas de visualização interativa, desenvolvido no início dos anos 1970 por Fisherkeller, Tukey, &amp; Friedman. [Um vídeo de demonstração sobreviveu!](https://www.youtube.com/watch?v=B7XoW2qiFUA)
- O conceito de _brushing &amp; linking_ foi formalizado por Becker, Cleveland, &amp; Wilks no artigo de 1987 [Gráficos Dinâmicos para a Análise de Dados](https://scholar.google.com/scholar?cluster=14817303117298653693).
- Para um resumo abrangente das técnicas de interação para visualização, veja [Dinâmicas Interativas para Análise Visual](https://queue.acm.org/detail.cfm?id=2146416) de Heer &amp; Shneiderman.
- Por fim, para um estudo sobre o que torna a interação eficaz, leia o clássico artigo [Interfaces de Manipulação Direta](https://scholar.google.com/scholar?cluster=15702972136892195211) de Hutchins, Hollan, &amp; Norman.

#### Apêndice: Na Representação do Tempo

Anteriormente, observamos um pequeno aumento no número de filmes em 1969 ou 1970. De onde vem esse aumento? E por que 1969 _ou_ 1970? A resposta decorre de uma combinação de dados ausentes e da forma como o seu computador representa o tempo.

Internamente, datas e horários são representados em relação à [época UNIX](https://en.wikipedia.org/wiki/Unix_time), na qual o tempo "zero" corresponde exatamente à meia-noite de 1º de janeiro de 1970 no [horário UTC](https://en.wikipedia.org/wiki/Coordinated_Universal_Time), que segue ao longo do [meridiano primário](https://en.wikipedia.org/wiki/Prime_meridian). Acontece que alguns filmes possuem datas de lançamento ausentes (`null`). Esses valores `null` são interpretados como tempo `0`, e, portanto, mapeados para 1º de janeiro de 1970 no horário UTC. Se você estiver nas Américas &ndash; e, portanto, em fusos horários "anteriores" &ndash;, esse instante exato corresponde a um horário anterior em 31 de dezembro de 1969 no seu fuso horário local. Por outro lado, se você estiver próximo ou a leste do meridiano principal, a data no seu horário local será 1º de janeiro de 1970.

A lição? Sempre desconfie dos seus dados e esteja atento ao fato de que a forma como os dados são representados (seja como datas e horários, números de ponto flutuante, latitudes e longitudes, _etc._) pode, às vezes, introduzir artefatos que afetam a análise!